In [1]:
! pip install pandoc

ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files\PostgreSQL\16\ssl\certs\ca-bundle.crt



# CONDITIONAL GAN

## NOTA
Como esta práctica se escapaba a mi comprensión , para hacerla me he ayudado de recursos en internet como documentación, paginas web, y asistentes de IA

En esta práctica deberéis implementar una GAN condicional con el dataset FashionMNIST.

Podéis basaros en este ejemplo: https://keras.io/examples/generative/conditional_gan/.

Lo único que debéis cambiar es la entrada, tanto del G como del D, para incluir la clase que queréis asociar con la imagen generada.


In [ ]:
# montamos la unidad drive donde tenemos los datos en la carpeta drive/My Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# importamos las librerías necesarias
import numpy as np
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
import matplotlib.pyplot as plt
from keras.utils import plot_model

## DEFINIENDO EL DISCRIMINADOR: MODELO FUNCIONAL

In [6]:
# definimos el discriminador: en este caso va a ser convolucional
def define_discriminator(in_shape=(28, 28, 1), n_classes=10):

    # creamos la entrada de la información condicional
    in_label = Input(shape=(1,))                            # el input de la etiqueta
    mapping_cond_inf = Embedding(n_classes, 50)             # embedding de input categórico 
    
    """
    # debemos añadir una capa densa que nos transforme el escalar que representa
    # nuestra clase a una imagen de (28, 28) para luego poder concatenar esa 
    # información con la imagen
    """
    n_nodes = in_shape[0]*in_shape[1]
    mapping_cond_inf = Dense(n_nodes)(in_label)
    mapping_cond_inf = LeakyReLU(alpha= 0.1)(mapping_cond_inf)
    mapping_cond_inf = Reshape((in_shape[0], in_shape[1], 1))(mapping_cond_inf)

   
    in_image = Input(shape=in_shape)                                  # entrada de la imagen de 28x28
    merge = Concatenate()([in_image, mapping_cond_inf])               # concatenamos la info condicional con la imagen

    # downsample
    fe = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Flatten()(fe)
    fe = Dropout(0.4)(fe)
    out_layer = Dense(1, activation='sigmoid')(fe) 
    
    model = Model([in_image, in_label], out_layer)               # definimos el modelo
    opt = Adam(learning_rate=0.0002, beta_1=0.5)                            # optimizador adam

    # compilamos
    model.compile(loss='binary_crossentropy',                    # output binario real o fake                    
                  optimizer=opt, metrics=['accuracy'])           # metrica: accuracy
    return model

discriminador = define_discriminator()
discriminador.summary()
plot_model(define_discriminator(), to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 dense_5 (Dense)             (None, 784)                  1568      ['input_7[0][0]']             
                                                                                                  
 leaky_re_lu_7 (LeakyReLU)   (None, 784)                  0         ['dense_5[0][0]']             
                                                                                                  
 input_8 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                            

## DEFINIENDO EL GENERADOR: MODELO FUNCIONAL 

In [19]:
# definimos el generador
def define_generator(latent_dim= 100, n_classes=10):
    
    # creamos la entrada de la información condicional
    in_label = Input(shape=(1,))
    emb = Embedding(n_classes, 50)(in_label)
    n_nodes = 7 * 7
    mapping_cond_inf = Dense(n_nodes)(emb)
    mapping_cond_inf = Reshape((7, 7, 1))(mapping_cond_inf)


    in_lat = Input(shape=(latent_dim,))

    # número de dimensiones para poder tratar con él con nuestra CNN
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)

    
    merge = Concatenate()([gen, mapping_cond_inf])                           # concatenamos el código latente y la información
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)  # aumentamos a 14x14
    gen = LeakyReLU(alpha=0.2)(gen)                                          # activación
    
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)    # aumentamos a 28x28
    gen = LeakyReLU(alpha=0.2)(gen)                                          # activación 
    
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)     # salida del modelo (una imagen en escala de grises de 28x28)
    model = Model([in_lat, in_label], out_layer)                             # definimos el modelo
    return model

define_generator().summary()
plot_model(define_discriminator(), to_file='generator_plot.png', show_shapes=True, show_layer_names=True)

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_42 (InputLayer)       [(None, 100)]                0         []                            
                                                                                                  
 input_41 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 dense_40 (Dense)            (None, 6272)                 633472    ['input_42[0][0]']            
                                                                                                  
 embedding_20 (Embedding)    (None, 1, 50)                500       ['input_41[0][0]']            
                                                                                           

## DEFINIENDO LA GAN, MODELO FUNCIONAL

In [14]:
# definimos el modelo GAN combinando generador y discriminador, para entrenar el generador
def define_gan(g_model, d_model):

    d_model.trainable = False
    gen_noise, gen_label = g_model.input
    gen_output = g_model.output
    
    gan_output = d_model([gen_output, gen_label])
    model = Model([gen_noise, gen_label], gan_output)
    
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    
    return model

plot_model(define_gan(), to_file='generator_plot.png', show_shapes=True, show_layer_names=True)

## CARGANDO - GENERANDO IMÁGENES REALES

In [ ]:
# definimos las funciones para cargar el MNIST
def load_real_samples():
    from tensorflow.keras.datasets.fashion_mnist import load_data
    
    (trainX, trainY), (_, _) = load_data()     # cargamos el dataset - ESTA VEZ QUEREMOS LAS IMÁGENES Y LAS CLASES
    X = np.expand_dims(trainX, axis=-1)        # expandimos la dimensión del batch
    X = X.astype('float32')                    # convertimos a float32
    X = (X - 127.5) / 127.5                    # escalamos entre -1 y 1
    return X, trainY                           # devolvemos tanto las imágenes como las clases

"""
# nos creamos una función que nos devuelva n_samples del dataset (imagen, clase) 
# y generamos las etiquetas de entrenamiento GAN: 1 
"""
def generate_real_samples(dataset, n_samples):
    
    images, labels = dataset                               # separamos las imágenes de las etiquetas
    ix = np.random.randint(0, images.shape[0], n_samples)  # seleccionamos n_samples muestras aleatoriamente
    X, labels = images[ix], labels[ix]                     # las cogemos junto a su correspondiente clase
    y = np.ones((n_samples, 1))                            # generamos las etiquetas para entrenar la GAN (1)

    return (X, labels), y

## GENERANDO IMÁGENES FALSAS

In [ ]:
# generamos los vectores latentes que introduciremos al generador
def generate_latent_points(latent_dim, batch_size, n_classes=10):
    # batch_size es el número de elementos por batch 
    # latent_dim es la dimensión del vector latente
    
    x_input = np.random.randn(latent_dim * batch_size)   # generamos un vector de batch_size * latent_dim números aleatorios
    z_input = x_input.reshape(batch_size, latent_dim)    # redimensionamos el vector para que tenga un tamaño (batch_size, latent_dim)
    labels = np.random.randint(low= 0,
                               high= n_classes, 
                               size= batch_size)
    return z_input, labels                               # el vector de ruido como la "clase": z_input y labels
 
# creamos datos fake con el generador (dinero falsificado)
def generate_fake_samples(g_model, latent_dim, n_samples): 
     
    z_input, labels_input = generate_latent_points(latent_dim, n_samples) # usamos la función anterior para generar los vectores latentes
    images = g_model.predict([z_input, labels_input])                     # le introducimos los vectores latentes y las clases al generador
    y = np.zeros(shape= (n_samples, 1))                                   # etiqueta 0 porque utilizaremos esta función para el discriminador
    """    
    # debemos devolver, por un lado, images y labels_inputs (para condicionar la GAN), 
    # y por el otro, la etiqueta que le asignamos: en este caso, al estar
    # entrenando el generador, etiqueta = 0
    """
    return [images, labels_input], y

In [ ]:
# función para guardar las imágenes generadas
def save_plot(examples, epoch, n=10, figsize=(20, 20)):
    examples = (examples * 127.5) + 127.5
    plt.figure(figsize=figsize)
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
    # guardamos las imágenes
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()

## DEFINIENDO EL ENTRENAMIENTO

In [20]:
# función para entrenar la GAN: el discriminador y el generador
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    for epoch in range(n_epochs):               # bucle para las epochs
        for batch in range(bat_per_epo):        # bucle para los batch
            
            """
            # vamos a separar las pérdidas del discriminador
            # cuando le metemos imágenes reales y cuando le metemos imágenes
            # fake para ver cómo lo hace con cada tipo
            # lo ideal es que llegue a un 50% de acc en cada uno
            """

            
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)              # preparamos los datos reales
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)                      # actualizamos el discriminador
            
            [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)  # generamos datos falsos
            d_loss2, _ = d_model.train_on_batch([X_fake, labels_fake], y_fake)                      # actualizamos el discriminador
			
             
            
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)   # preparamos los puntos en el espacio latente:
            
            """
            # creamos etiquetas invertidas para el generador: utilizamos el D(x) 
            # para que piense que las muestras que le introducimos son reales, y
            # en caso de que diga que no son reales, aprovechamos la información
            # de sus gradientes para actualizar el G(z) para que la próxima vez
            # los datos generados por G(z) sean más parecidos a los reales
            """
            y_gan = np.ones( shape= (n_batch, 1))     # un vector columna del tamaño del batch_size con todo unos
            
            """
            # entrenamos el generador de forma que actualice
            # sus pesos usando los gradientes del discriminador
            # en este modelo (gan_model) el discriminador está
            # congelado, por lo que no se actualizan sus pesos: 
            # no queremos "untar" a nuestro policía, lo que queremos 
            # es fabricar dinero más realista.
            """
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)

            # mostramos el progreso
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % 
                  (epoch+1, batch+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluamos el desempeño del modelo cada 10 épocas
        if (epoch+1) % 10 == 0 or epoch == 0:
            # preparamos los datos reales
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            # evaluamos el discriminador con datos reales
            _, acc_real = d_model.evaluate([X_real, labels_real], y_real, verbose=0)
            # preparamos ejemplos fake
            [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
            # evaluamos el discriminador con datos fake
            _, acc_fake = d_model.evaluate([X_fake, labels_fake], y_fake, verbose=0)
            # mostramos cómo de bueno es nuestro policía
            print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
            # guardamos las imágenes generadas
            save_plot(X_fake, epoch)
            # guardamos el generador para tenerlo disponible más tarde
            filename = 'cgan_generator_model_%03d.h5' % (epoch + 1)
            g_model.save(filename)

## CREANDO Y ENTRENANDO EL MODELO CONDITIONAL GAN

In [ ]:

latent_dim = 100                         # size of the latent space
d_model = define_discriminator()         # create the discriminator
g_model = define_generator(latent_dim)   # create the generator
gan_model = define_gan(g_model, d_model) # create the gan
dataset = load_real_samples()            # load image data
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

In [ ]:
ls

In [ ]:
# los guardamos en nuestro drive para evitar tener que reejecutar cada vez
!mkdir drive/My\ Drive/7_cond_dcgan_fashionmnist
!cp *gen* drive/My\ Drive/7_cond_dcgan_fashionmnist/
!ls -lah drive/My\ Drive/7_cond_dcgan_fashionmnist/

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(plt.imread('generated_plot_e001.png'))
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(plt.imread('generated_plot_e010.png'))
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(plt.imread('generated_plot_e100.png'))
plt.show()

In [ ]:
# vamos a generar imágenes de una determinada clase
from tensorflow.keras.models import load_model

# cargamos el modelo
model = load_model('cgan_generator_model_100.h5')
# generate images
latent_points, labels = generate_latent_points(100, 100)
# specify labels
labels = np.asarray([x for _ in range(10) for x in range(10)])
# generate images
X  = model.predict([latent_points, labels])
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
save_plot(X, 100)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(plt.imread('generated_plot_e101.png'))
plt.show()